In [1]:
import os
import pandas as pd
import numpy as np
import json
from PIL import Image, ImageDraw
import cv2
import numpy as np
import ultralytics

## Load Split data

In [57]:
def split_train_test(dir_path, csv_file):
    with open(csv_file, 'r') as f:
        for line in f:
            if line.startswith('screen'):
                continue
            subpath, dtype = line.strip().split(',')
            try:
                old_path = os.path.join(dir_path, subpath)
                new_path = os.path.join(dir_path, dtype, subpath)
                os.makedirs(os.path.dirname(new_path), exist_ok=True)
                for file in os.listdir(old_path):
                    os.rename(os.path.join(old_path, file), os.path.join(new_path, file))
                os.rmdir(old_path)
            except Exception as e:
                # print(e)
                pass
                

In [58]:
split_train_test('./IASA_Champ_Final/app_data', './IASA_Champ_Final/train_test_split.csv')

In [64]:
def count_files(dir_path, extentions=['.json', '.png']):
    counts = {ext: 0 for ext in extentions}
    for _, _, files in os.walk(dir_path):
        for file in files:
            for ext in extentions:
                if file.endswith(ext):
                    counts[ext] += 1
    return counts

In [65]:
count_files('./IASA_Champ_Final/app_data_splitted/train'), count_files('./IASA_Champ_Final/app_data_splitted/test')

({'.json': 192, '.png': 282}, {'.json': 52, '.png': 76})

In [116]:
def copy_all_photos(dir_path, target_dir, flag='box'):
    os.makedirs(target_dir, exist_ok=True)
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith('.png') and flag in file:
                new_path = os.path.join(target_dir, file)
                os.system(f'cp {os.path.join(root, file)} {new_path}')

copy_all_photos('./IASA_Champ_Final/app_data_splitted/train', './IASA_Champ_Final/app_data_splitted/train_photos_box', 'box')
copy_all_photos('./IASA_Champ_Final/app_data_splitted/test', './IASA_Champ_Final/app_data_splitted/test_photos_box', 'box')

cp: target 'Out-1707213723.15_box.png' is not a directory
cp: target 'Out-1707213671.76_box.png' is not a directory
cp: target '5-1707227556.76_box.png' is not a directory
cp: target '5-1707227600.45_box.png' is not a directory
cp: target '5-1707227575.92_box.png' is not a directory
cp: target '5-1707227584.28_box.png' is not a directory
cp: target '5-1707227424.73_box.png' is not a directory
cp: target '5-1707227544.27_box.png' is not a directory
cp: target '5-1707227479.89_box.png' is not a directory
cp: target '5-1707227592.58_box.png' is not a directory
cp: target 'Store-1707173656.53_box.png' is not a directory
cp: target 'Store-1707174074.67_box.png' is not a directory
cp: target 'Store-1707174099.08_box.png' is not a directory
cp: target 'Store-1707174123.38_box.png' is not a directory
cp: target 'Store-1707174130.99_box.png' is not a directory
cp: target 'Mail-1707211184.38_box.png' is not a directory
cp: target 'Mail-1707211213.72_box.png' is not a directory
cp: target 'Mail-1

In [91]:
classes = pd.read_csv('./classes.csv')
classes.head()
COLORS = ['red', 'green', 'blue', 'yellow', 'orange', 'purple', 'pink', 'cany','darkblue', 'grey']
class_colors = {c: COLORS[min(i, len(COLORS)-1)] for i, c in enumerate(classes['class'])}
classes['color'] = classes.apply(lambda row: COLORS[min(int(row.name), len(COLORS)-1)], axis=1)
classes.to_csv('./classes.csv', index=False)
classes.head(10)

,class,count,color
0,AXStaticText,9425,red
1,AXGroup,7390,green
2,AXButton,5204,blue
3,AXImage,4471,yellow
4,AXTextField,4220,orange
5,AXRow,3444,purple
6,AXCheckBox,2495,pink
7,AXCell,1579,cany
8,AXLink,961,darkblue
9,AXHeading,394,grey


## Anotation of the images

In [112]:
CLASSES = pd.read_csv('./classes.csv')

def draw_boxes(draw, item):
    bbox = item.get('visible_bbox')
    if bbox is not None:
        x, y, width, height = bbox
        box = [x, y, x + width, y + height]
        color = CLASSES[CLASSES['class'] == item['role']]['color'].values[0]
        draw.rectangle(box, outline=color)   
    children = item.get('children', [])
    for child in children:
        draw_boxes(draw, child)
        
def annotate_image(image_path, data_path, output_path=None, flag='box'):
    with open(data_path, 'r') as f:
        data = json.load(f)
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    draw_boxes(draw, data)
    if output_path is None:
        output_path = image_path.replace('.png', f'_{flag}.png')
    image.save(output_path)

def annotate_images(dir_path, flag='box'):
    for root, dirs, files in os.walk(dir_path):
        if any(files.endswith('.json') for files in files):
            for file in files:
                if file.endswith(".json"):
                    data_path = os.path.join(root, file)
                if file.endswith(".png") and flag not in file:
                    image_path = os.path.join(root, file)
            annotate_image(image_path, data_path, flag=flag)


In [115]:
annotate_images('./IASA_Champ_Final/app_data_splitted/train')

In [ ]:
def draw_csv_bboxes(file, image):
    df = pd.read_csv(file, sep='\t')
    image = Image.open(image)
    draw = ImageDraw.Draw(image)
    for _, row in df.iterrows():
        bbox = eval(row['visible_bbox'])
        x, y, width, height = bbox
        box = [x, y, x + width, y + height]
        draw.rectangle(box, outline='red')
    image.save('your_updated_image.png')


In [ ]:
def csv_to_json(file, output_file):
    df = pd.read_csv(file, sep='\t')
    data = {}
    for _, row in df.iterrows():
        parent_id = row['pid']
        if parent_id not in data:
            data[parent_id] = {'children': []}
        element = {'role': row['role'], 'visible_bbox': eval(row['visible_bbox']), 'value': row['value'], 'description': row['description']}
        data[row['id']] = element
        data[parent_id]['children'].append(element)
    
    with open(output_file, 'w') as f:
        json.dump(data[0], f)
        
# csv_to_json('./test.tsv', './test.json')

In [ ]:
draw_csv_bboxes('./your_updated_csv_file2.tsv', './Almighty-1707228195.40.png')

In [105]:
data_dir = './IASA_Champ_Final/app_data'
app_name = 'Setapp'
screen_id = '1707127584'
element_path = os.path.join(data_dir, app_name, screen_id)

for file in os.listdir(element_path):
    if file.endswith(".json"):
        data_path = os.path.join(element_path, file)
    if file.endswith(".png"):
        image_path = os.path.join(element_path, file)
            
annotate_image(element_path)

## Preprocess jsons

In [32]:
def assign_ids_recursive(data, id_counter=1, parent_id=0):
    data['id'] = id_counter
    data['pid'] = parent_id
    data.pop('bbox')
    id_counter += 1
    for idx in range(len(data['children'])-1, -1, -1):
        child = data['children'][idx]
        if child['visible_bbox'] is None or child['role_description'] in ['close button', 'minimize button', 'full screen button']:
            del data['children'][idx]
        else:
            id_counter = assign_ids_recursive(child, id_counter, data['id'])
    return id_counter

def list_all_elements(data):
    elements = [{k: v for k, v in data.items() if k != 'children'}]
    for element in data['children']:
        elements += list_all_elements(element)
    return elements

def create_text_description(row):
    text = [row['name'] if str(row['name']) != 'None' else '']
    if str(row['description'])!='None':
        text.append(row['description'])
    return '|'.join(text)

In [33]:
model = ultralytics.YOLO('best.pt')
cat_names = model.names

def fix_json(node, image):
    """
    :param node: JSON of GUI
    :param image_screenshot: cv2 image of GUI
    :return: cv2 image
    """
    if node['visible_bbox'] is not None:
        x, y, w, h = node['visible_bbox']
        yolo_res = model(cv2.resize(image[y:y+h,x:x+w,::-1], (244, 244)), verbose=False)
        top5 = [cat_names[i] for i in yolo_res[0].probs.top5]
        if np.sum(image[y:y+h, x:x+w] - image[y,x]) != 0 and node['role'] in top5:
            # image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            node['visible_bbox'] = None

    if node['children']:
        for child in node['children']:
            fix_json(child, image)
    
    return None

def filter_json(data, image_path: str):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fix_json(data, image)
    return data

In [37]:
def preprocess_json_to_csv(data_path, output_path, image_path=None):
    with open(data_path, 'r') as file:
        data = json.load(file)    
    assign_ids_recursive(data)
    if image_path is not None:
        data = filter_json(data, image_path)
    df = pd.DataFrame(list_all_elements(data))
    df['description'] = df.apply(create_text_description, axis=1)
    
    df['value'][df['value']=='None'] = ''
    df['role'] = df['role'].apply(lambda x: x[2:])    
    df['center'] = df['visible_bbox'].apply(lambda x: [x[0]+x[2]/2, x[1]+x[3]/2] if x is not None else None)
    
    df = df[['id', 'pid', 'role','visible_bbox', 'center', 'value','description']]
    df.to_csv(output_path, sep='\t', index=False)

In [38]:
def prepare_all_tsvs(dir_path, inflag='', exflag='annotated'):
    for root, dirs, files in os.walk(dir_path):
        if any(files.endswith('.json') for files in files):
            for file in files:
                if file.endswith(".json") and inflag in file and exflag not in file:
                    data_path = os.path.join(root, file)
                if file.endswith(".png") and 'box' not in file:
                    image_path = os.path.join(root, file)
            output_file_name = '_'.join(root.split('/')[-2:]) + '.tsv'
            output_path = os.path.join(root, output_file_name)
            preprocess_json_to_csv(data_path, output_path, image_path)


In [39]:
data_path = './com.onmyway133.Almighty-setapp.json'
image_path = './Almighty-1707228195.40.png'
output_path = './your_updated_csv_file.tsv'
preprocess_json_to_csv(data_path, output_path, image_path)

/tmp/ipykernel_67799/2285943580.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value'][df['value']=='None'] = ''


In [ ]:
prepare_all_tsvs('./IASA_Champ_Final/app_data')